In [1]:
import torch.nn as nn
import torch

In [3]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class VGG(nn.Module):
    def __init__(self,vgg_name,num_classes=7):
        super(VGG,self).__init__()
        self.features= self._make_layers(cfg[vgg_name]) # 함수 앞 _는 private
        self.classifier = nn.Linear(512, num_classes)
        
    def forward(self,x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    def _make_layers(self,cfg):
        layers=[]
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2,stride=2)]
            else :
                layers +=[nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                          nn.BatchNorm2d(x),
                          nn.ReLU(inplace=True)]
                in_channels =x
        layers+=[nn.AvgPool2d(kernel_size=1,stride=1)]
        return nn.Sequential(*layers)
        

In [4]:
def test():
    net = VGG('VGG11')
    x = torch.randn(2,3,48,48)
    y = net(x)
    print(y.size())

In [5]:
test()

torch.Size([2, 7])


In [6]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [7]:
input_data=np.load("input_data_emotion.npy")
label=np.load("label.np.npy")

In [8]:
label=np.array(label)
input_data= (input_data*(1/256)-0.5)/0.5

In [9]:
input_data=torch.FloatTensor(input_data)
label=torch.LongTensor(label)

In [10]:
input_data=input_data.transpose(1,3)

In [11]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
dataset = TensorDataset(input_data,label)

In [12]:
trainset,valset=torch.utils.data.random_split(dataset,[30887,5000])
trainset,testset=torch.utils.data.random_split(trainset,[25887,5000])

partition={'train':trainset,'val':valset,'test':testset}

In [13]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=2048,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [14]:
def test(net):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=2048, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [15]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=2048, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [16]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [17]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net = VGG('VGG11')
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(100):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

  0%|          | 0/13 [00:00<?, ?it/s]

time : 7.37, train_loss : 2.10, train_acc : 23.39,val_acc : 20.48,val_loss : 4.56


  0%|          | 0/13 [00:00<?, ?it/s]

time : 7.08, train_loss : 1.80, train_acc : 25.55,val_acc : 26.30,val_loss : 1.80


  0%|          | 0/13 [00:00<?, ?it/s]

...

time : 7.39, train_loss : 0.00, train_acc : 99.83,val_acc : 61.88,val_loss : 2.50


  0%|          | 0/13 [00:00<?, ?it/s]

time : 7.34, train_loss : 0.00, train_acc : 99.83,val_acc : 61.12,val_loss : 2.60


100%|██████████| 3/3 [00:00<00:00,  4.27it/s]

time : 7.37, train_loss : 0.00, train_acc : 99.84,val_acc : 61.70,val_loss : 2.60


In [18]:
np.save("/home/lab/양창희/Emotion Project/test_list_file/VGG_train_loss.npy",train_loss)
np.save("/home/lab/양창희/Emotion Project/test_list_file/VGG_val_loss.npy",val_loss)
np.save("/home/lab/양창희/Emotion Project/test_list_file/VGG_train_acc.npy",train_acc)
np.save("/home/lab/양창희/Emotion Project/test_list_file/VGG_val_acc.npy",val_acc)